Objective: Utilize Python to connect with OpenAI's LLM for summarizing webpage content. The project focuses on healthcare-related websites such as CMS, NCQA, and Elevance Health. Key libraries employed include BeautifulSoup, dotenv, OpenAI, and requests. This approach streamlines the process of screening relevant webpages, with the summarized outputs being standardized enough for potential use in creating a research-oriented website review database.

In [2]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY','openai_api_key')
openai = OpenAI()

In [9]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [21]:
cms = Website("https://cms.gov")
print(cms.title)
print(cms.text)

Home - Centers for Medicare & Medicaid Services | CMS
Skip to main content
An official website of the United States government
Here's how you know
Here's how you know
Official websites use .gov
A
.gov
website belongs to an official government organization in the United States.
Secure .gov websites use HTTPS
A
lock
(
)
or
https://
means you've safely connected to the .gov website. Share sensitive information only on official, secure websites.
Centers for Medicare & Medicaid Services
CMS Newsroom
Header
About CMS
Newsroom
Data & Research
Search CMS.gov
Search
Search
Popular terms
Physician Fee Schedule
Local Coverage Determination
Medically Unlikely Edits
Telehealth
Covid-19
CMS.gov main menu
Medicare
Back to main menu
section title h2
Enrollment & renewal
Back to
menu
section title h3
Original Medicare (Part A and B) Eligibility and Enrollment
Annual Medicare Participation Announcement
Providers & suppliers
Medicare Managed Care Eligibility and Enrollment
Part D Eligibility and Enrollme

In [11]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [12]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [13]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [14]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [25]:
# Not human readable
summarize("https://www.elevancehealth.com")

'# Elevance Health Overview\n\nElevance Health is a healthcare organization dedicated to supporting health at every life stage by providing a comprehensive array of health plans and clinical, behavioral, pharmacy, and complex-care solutions aimed at promoting whole health. They prioritize improving health outcomes through innovations in healthcare delivery, aiming to integrate and advance health beyond traditional healthcare settings.\n\n## Key Focus Areas\n- **Whole Health Approach**: They advocate for a holistic view of health, considering physical, behavioral, and social factors.\n- **Community and Equity**: Elevance Health emphasizes community health and health equity, addressing social determinants of health to improve overall outcomes.\n- **Consumer-Centered Healthcare**: The organization seeks to create a healthcare system that is centered around consumer needs, promoting personalized and easily navigable experiences.\n\n## Recent News & Initiatives\n- **Social Needs Investment 

In [16]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [26]:
display_summary("https://www.elevancehealth.com")

# Elevance Health Overview

Elevance Health is dedicated to supporting health at every stage of life by offering a range of health plans and integrated solutions focusing on clinical, behavioral, pharmacy, and complex care, all aimed at promoting whole health. The company emphasizes a commitment to improving health outcomes through innovative approaches and community engagement.

## Key Areas of Focus
- **Whole Health Approach**: Redefining health by integrating physical, behavioral, and social factors.
- **Community Health**: Enhancing health equity and addressing social determinants of health.
- **Consumer-Centered Healthcare**: Transforming the healthcare experience to make it more user-friendly and personalized.

## Recent Developments
- **Social Needs Investment Lab**: In partnership with HealthBegins, Elevance Health launched a lab to evaluate programs aimed at improving health outcomes related to social needs.
- **Hurricane Recovery Efforts**: Following Hurricane Helene's impact, Elevance Health associates participated in recovery activities, highlighting the company’s community support initiatives.
- **Whole Health Index**: This tool allows for better identification of individual health needs by analyzing diverse health data.

## News Highlights
- **Impact of Food Insecurity**: A feature discussing the rising issue of food insecurity on college campuses, illustrating the broader implications of health-related challenges.
- **2023 Progress Report**: A publication detailing the company's collaborative approach with care providers to enhance health outcomes across communities.

Elevance Health aims to elevate and advance health beyond traditional healthcare services, focusing on holistic and community-oriented solutions.

In [18]:
display_summary("https://ncqa.org")

# Summary of NCQA Website

The National Committee for Quality Assurance (NCQA) focuses on improving healthcare quality through various accreditation programs and initiatives. The organization offers a range of services, including health plan accreditation, recognition for patient-centered medical homes, and programs to assess health equity and utilization management. 

### Key Programs and Initiatives
- **Accreditation Programs**: NCQA accredits healthcare organizations and practices, including health plans, specialty pharmacies, and physician practices.
- **HEDIS Measures**: NCQA manages the Healthcare Effectiveness Data and Information Set (HEDIS), which provides performance metrics to assess the quality of care.
- **Health Equity**: The organization promotes health equity through projects that leverage race and ethnicity data in quality measures.

### Upcoming Events
- **Health Innovation Summit**: Scheduled for October 31 to November 2, 2024, in Nashville, TN.

### Recent News Highlights
- **Health Care Quality Week (December 2024)**: Celebrated by spotlighting quality innovators.
- **Health Equity Partnership**: Arizona selected as the first partner to support providers with Health Equity Accreditation.
- **2024 Health Plan Ratings Unveiled**: NCQA released its ratings for 2024, showcasing the performance of accredited health plans.
- **Launch of Virtual Care Accreditation**: A new accreditation program aimed at enhancing virtual healthcare services.

### Educational Resources
- NCQA provides online seminars, webinars, and on-demand training to enhance insights into their programs and facilitate professional development.

This website serves as a hub for healthcare professionals seeking to improve care quality and access invaluable resources related to best practices, data, and accreditation standards.

In [19]:
display_summary("https://cms.gov")

# Summary of Centers for Medicare & Medicaid Services (CMS) Website

The **Centers for Medicare & Medicaid Services (CMS)** is a U.S. government agency that oversees the nation's health care programs, specifically Medicare, Medicaid, and the Children's Health Insurance Program (CHIP). The website offers a wealth of information about enrollment processes, coverage options, regulations, and guidance for both beneficiaries and health care providers.

## Key Features

- **Enrollment & Coverage**: Information on various Medicare parts (A, B, C, D), Medicaid, and CHIP eligibility and enrollment processes.
- **Regulations & Guidance**: Detailed manuals, guidelines, and legislative information related to health care regulation.
- **Quality & Safety**: Tools and initiatives for improving health care quality and establishing safety standards.
- **Fraud Prevention**: Resources aimed at combating healthcare fraud and promoting compliance.

## Recent News and Announcements

1. **Inflation Reduction Act**: New policies under this law are designed to save money for Medicare recipients and improve access to affordable treatments.
2. **Medicaid Renewals**: As states return to normal operations, CMS is focused on ensuring individuals maintain their coverage.
3. **Innovation Center Update**: Ongoing support for innovative health care payment and service delivery models.
4. **No Surprise Billing**: New rules enacted to protect consumers from unexpected medical bills.
5. **Nursing Home Resources**: Updated policy information and resources for quality care in nursing homes.

## Goals and Initiatives

- **Advance Health Equity**: Address health disparities to improve health outcomes for all populations.
- **Expand Access**: Enhance the Affordable Care Act to offer broader health coverage.
- **Drive Innovation**: Promote person-centered care and address health system challenges through innovation.

This website serves as a central platform for individuals seeking information on Medicare, Medicaid, health insurance marketplaces, and resources for health care providers.

In [20]:
display_summary("https://wsj.com")

# Summary of The Wall Street Journal Website

The Wall Street Journal (WSJ) is a prominent English-language news source covering a broad range of topics including world news, business, politics, technology, markets, personal finance, health, and lifestyle. It provides in-depth reporting, expert opinions, and analysis.

## Main Sections
- **World News**: Coverage of global events across regions such as Africa, Asia, Europe, and the Americas.
- **Business**: Focus on various industries, economic trends, and corporate news.
- **Politics**: Insights into U.S. elections, national security, and policy matters.
- **Technology**: Articles on advancements in AI, biotech, and personal technology sectors.
- **Markets & Finance**: Analysis of stocks, banking, regulation, and investing trends.
- **Opinion**: A collection of editorials, commentary from columnists, and analysis on various issues.
- **Arts & Lifestyle**: Reviews and features on books, films, food, fitness, and travel.

## Featured Articles
- **How Barron Trump Connected His Father to the Manosphere** explores the influence of social movements on political figures.
- **Qatar Pauses Efforts to Mediate Stalled Gaza Cease-Fire** discusses international diplomatic efforts concerning the Gaza conflict.
- **A Democrat Ponders a ‘Thumping Rebuke’** examines political strategies ahead of an upcoming electoral cycle.

## Podcasts
The WSJ offers a variety of podcasts, including discussions on current events and expert insights into finance and markets.

This summary captures the essence of The Wall Street Journal's website and highlights its core offerings without delving into navigational aspects.